In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
# read in relevant tables, change index to 1-index to match dataset
# i'm also printing the first row of each DataFrame as an easy reference point to columns

races = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/races.csv")
races.index = races.index + 1
print("races")
display(races.head(1))

drivers = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/drivers.csv")
drivers.index = drivers.index + 1
print("drivers")
display(drivers.head(1))

driver_standings = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/driver_standings.csv")
driver_standings.index = driver_standings.index + 1
print("driver_standings")
display(driver_standings.head(1))

constructors = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/constructors.csv")
constructors.index = constructors.index + 1
print("constructors")
display(constructors.head(1))

constructor_standings = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/constructor_standings.csv")
constructor_standings.index = constructor_standings.index + 1
print("constructor_standings")
display(constructor_standings.head(1))

results = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2021/2021-09-07/results.csv")
results.index = results.index + 1
print("results")
display(results.head(1))

races


,raceId,year,round,circuitId,name,date,time,url
1,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...


drivers


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
1,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


driver_standings


...

,constructorId,constructorRef,name,nationality,url
1,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren


constructor_standings


,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
1,1,18,1,14.0,1,1,1


results


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
1,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [3]:
# merge name columns for later merging into tables
drivers["driverName"] = drivers["forename"] + " " + drivers["surname"]

# rename columns for same reason
driver_standings.rename(columns={"position": "driverPosition", "points": "driverPoints"}, inplace=True)
constructors.rename(columns={"name": "constructorName"}, inplace=True)
constructor_standings.rename(columns={"position": "constructorPosition", "points": "constructorPoints"}, 
                             inplace=True)

In [4]:
# I'm going to focus on data from the year that Schumacher started racing in F1
years = races[races["year"] >= 1980]["year"].unique()
last_raceId_per_year = races.groupby("year")["raceId"].max().reindex(years).to_numpy()
last_raceId_per_year

array([  17,   35,   52,   70,   89,  107,  123,  140,  157,  174,  190,
        206,  223,  239,  256,  272,  288,  304,  320,  336,  355,  371,
        387,  403,  419,  435,  451,  466,  482,  497,  511,  859,  879,
        899,  918,  945,  968,  988, 1009, 1030, 1047, 1073])

In [5]:
constructor_standings.head()

,constructorStandingsId,raceId,constructorId,constructorPoints,constructorPosition,positionText,wins
1,1,18,1,14.0,1,1,1
2,2,18,2,8.0,3,3,0
3,3,18,3,9.0,2,2,0
4,4,18,4,5.0,4,4,0
5,5,18,5,2.0,5,5,0


In [6]:
# remove all rows from `races` where 'raceId' not in `last_raceId_per_year`
# filtered_races = races[races["raceId"].isin(last_raceId_per_year)]

# merge `driver_standings` stats with `filtered_races`, use inner join to only retain relevant rows
driver_details = pd.merge(
    driver_standings, filtered_races[["raceId", "year"]],
    on="raceId", how="inner"
)

# add 'constructorId' from `results`
driver_results = pd.merge(
    driver_details, results[["raceId", 'driverId', "constructorId"]],
    on=["raceId", 'driverId'], how="inner"
)

# add 'driverName' by merging with `drivers`
driver_results = pd.merge(
    driver_results, drivers[['driverId', 'driverName']],
    on='driverId', how="inner"
)

# add constructorPoints and constructorPosition and merge with constructor standings
driver_constructor_details = pd.merge(
    driver_results,
    constructor_standings[["raceId", "constructorId", "constructorPosition", "constructorPoints"]],
    on=["raceId", "constructorId"], how="inner"
)

# finally, add constructorName
final_table = pd.merge(
    driver_constructor_details,
    constructors[["constructorId", "constructorName"]],
    on="constructorId", how="inner"
)

# select and rename columns
# final_table = final_table[['driverName', 'year', 'position', 'constructorName', 'constructorPosition']]
final_table.drop(columns=["positionText"], inplace=True)

# add a label to know which races were the final raes of the season
final_table["isFinale"] = final_table["raceId"].isin(last_raceId_per_year)

final_table[(final_table["year"] == 2008) & (final_table["isFinale"] == True)]

NameError: name 'filtered_races' is not defined

In [ ]:
# further cleanup of data

# sort by raceId and points to assign positions within each race
final_table.sort_values(by=["raceId", "driverPoints", "constructorPosition"], 
                        ascending=[True, False, True], inplace=True)

# recreate driverPosition based on sorted points (driver with max points gets position 1)
# note that those with 0 points have been ordered by myself according to their constructor's position
final_race_table = final_table.loc[final_table["isFinale"] == True]
final_race_table['driverPosition'] = final_race_table.groupby('raceId').cumcount() + 1

# remove drivers who never scored a point (for diagram clarity)
drivers_score = final_race_table.groupby("driverId")["driverPoints"].sum()

drivers_who_score = drivers_score[drivers_score > 0].index
final_table = final_table[final_table["driverId"].isin(drivers_who_score)]

final_table[(final_table["year"] == 2008) & (final_table["isFinale"] == False)]

In [ ]:
final_table.isna().sum()

In [16]:
final_table.to_csv("f1_results_clean.csv")